<a href="https://colab.research.google.com/github/Pa345-ai/Ultra-premium-ai-lead-generator-/blob/main/Ultra-Premium-ai-Lead-generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Install required packages ---
!pip install --quiet streamlit pyngrok pandas numpy faker plotly openai

# --- Imports ---
import os
import streamlit as st
import pandas as pd
import numpy as np
from faker import Faker
from pyngrok import ngrok
import plotly.express as px
import threading
import time
from openai import OpenAI

fake = Faker()

# --- Set ngrok authtoken ---
NGROK_AUTHTOKEN = "32JVcF0u9Wdf7vhliGN1iNCCb7C_5DUp2ewNSr5bkrvgVjJD8"
os.environ["NGROK_AUTHTOKEN"] = NGROK_AUTHTOKEN
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# --- OpenAI client (replace with your API key) ---
os.environ["OPENAI_API_KEY"] = "sk-your-real-openai-key"
client = OpenAI()

# --- Generate real AI features per lead ---
def generate_ai_features(lead):
    prompt = f"""
    You are an AI Lead Generation Assistant.
    Generate 28 premium outreach & sales features for the following lead:

    Name: {lead['Name']}
    Company: {lead['Company']}
    Job Title: {lead['Job Title']}
    Revenue: {lead['Revenue']}
    Company Size: {lead['Company Size']}
    Priority Score: {lead['Priority Score']}

    Return the result strictly as a JSON object with exactly these keys:
    ["email_subject","email_body","linkedin_message","sms_message",
    "followups","best_time_to_contact","predicted_reply_rate","objection_handling",
    "campaign_insights","lead_scoring","roi_estimate","proposal_suggestion",
    "meeting_schedule_suggestion","multi_language_outreach","competitor_insights",
    "automated_followups","predictive_conversion","ai_content_recommendations",
    "lead_enrichment","multi_channel_notifications","smart_tagging",
    "ai_reporting","ab_testing","predictive_roi","ai_scheduler",
    "personalized_campaigns","ai_analytics","crm_sync"]
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You return structured JSON only."},
                  {"role": "user", "content": prompt}],
        temperature=0.7
    )

    try:
        return eval(response.choices[0].message.content)
    except Exception:
        return {"error": "Invalid AI response", "raw": response.choices[0].message.content}

# --- Streamlit App ---
def run_streamlit():
    st.set_page_config(page_title="🚀 Ultra Premium AI Lead Generation Tool", layout="wide")
    st.title("🚀 Ultra Premium AI Lead Generation Tool")

    # 1️⃣ Advanced Lead Discovery
    st.header("1️⃣ Advanced Lead Discovery")
    num_leads = st.slider("Number of leads to generate:", 5, 20, 5)
    leads = pd.DataFrame({
        "Name": [fake.name() for _ in range(num_leads)],
        "Email": [fake.email() for _ in range(num_leads)],
        "Company": [fake.company() for _ in range(num_leads)],
        "Job Title": [fake.job() for _ in range(num_leads)],
        "Revenue": [np.random.randint(50_000, 10_000_000) for _ in range(num_leads)],
        "Company Size": np.random.choice(["Startup","SMB","Mid-Market","Enterprise"], num_leads),
        "Priority Score": np.random.randint(50,100,num_leads)
    })
    st.dataframe(leads)

    # 2️⃣ AI Outreach & 28 Premium Features
    st.header("2️⃣ AI Outreach & Premium Features")
    for i, lead in leads.iterrows():
        st.subheader(f"Lead {i+1}: {lead['Name']}")
        features = generate_ai_features(lead)
        st.json(features)
        time.sleep(0.5)

    # 3️⃣ Analytics & Insights
    st.header("3️⃣ Analytics & Insights")
    fig = px.bar(leads, x="Company", y="Revenue", title="Revenue per Company")
    st.plotly_chart(fig)

    # 4️⃣ Export & CRM Integration
    st.header("4️⃣ Export & CRM Integration")
    st.download_button("Download Leads CSV", data=leads.to_csv(index=False), file_name="leads.csv")

# --- Save Streamlit app to file ---
with open("/content/app.py", "w") as f:
    f.write("""
import streamlit as st, pandas as pd, numpy as np, time
from faker import Faker
import plotly.express as px
from openai import OpenAI
fake = Faker()
client = OpenAI()

def generate_ai_features(lead):
    prompt = f\"\"\"Generate 28 premium features in JSON for this lead:
    Name: {lead['Name']}
    Company: {lead['Company']}
    Job Title: {lead['Job Title']}
    Revenue: {lead['Revenue']}
    Company Size: {lead['Company Size']}
    Priority Score: {lead['Priority Score']}\"\"\"
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"system","content":"Return JSON only."},
                  {"role":"user","content":prompt}],
        temperature=0.7
    )
    try:
        return eval(response.choices[0].message.content)
    except Exception:
        return {"error":"Invalid AI response","raw":response.choices[0].message.content}

st.set_page_config(page_title="Ultra Premium AI Lead Generator", layout="wide")
st.title("🚀 Ultra Premium AI Lead Generation Tool")
num_leads = st.slider("Number of leads to generate:", 5, 20, 5)
leads = pd.DataFrame({
    "Name": [fake.name() for _ in range(num_leads)],
    "Email": [fake.email() for _ in range(num_leads)],
    "Company": [fake.company() for _ in range(num_leads)],
    "Job Title": [fake.job() for _ in range(num_leads)],
    "Revenue": [np.random.randint(50_000, 10_000_000) for _ in range(num_leads)],
    "Company Size": np.random.choice(["Startup","SMB","Mid-Market","Enterprise"], num_leads),
    "Priority Score": np.random.randint(50,100,num_leads)
})
st.dataframe(leads)
for i, lead in leads.iterrows():
    st.subheader(f"Lead {i+1}: {lead['Name']}")
    st.json(generate_ai_features(lead))
    time.sleep(0.5)
fig = px.bar(leads, x="Company", y="Revenue", title="Revenue per Company")
st.plotly_chart(fig)
st.download_button("Download Leads CSV", data=leads.to_csv(index=False), file_name="leads.csv")
""")

# --- Launch Streamlit in background ---
threading.Thread(target=lambda: os.system("streamlit run /content/app.py &")).start()

# --- Start ngrok tunnel and print public URL ---
public_url = ngrok.connect(8501)
print(f"🔗 Public Streamlit URL via ngrok: {public_url}")